In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
from datetime import datetime
from dateutil.parser import parse


def read_prep_data(file):
    df = file
    # missining selected
    df = df[df['Selected'] != 'missing']  # make sure that we have data for selected attribute

    # CreditScore handeling
    df.CreditScore.replace(0.0, np.nan, inplace=True)
    df['CreditScore'].fillna((df['CreditScore'].median()), inplace=True)

    # MonthlyCost handeling
    df.MonthlyCost.replace(0.0, np.nan, inplace=True)
    df['MonthlyCost'].fillna((df['MonthlyCost'].median()), inplace=True)

    # NumberOfTerms handeling
    df.NumberOfTerms.replace(0.0, np.nan, inplace=True)
    df['NumberOfTerms'].fillna((df['NumberOfTerms'].median()), inplace=True)

    # FirstWithdrawalAmount handeling
    df.FirstWithdrawalAmount.replace(0.0, np.nan, inplace=True)
    df['FirstWithdrawalAmount'].fillna((df['FirstWithdrawalAmount'].median()), inplace=True)

    # map Selected to 1(signed), and 0(not signed)
    df['Selected'] = df['Selected'].map({'True': 1, 'False': 0})
    
    # map label to 1(signed), and 0(not signed)
    df['label'] = df['label'].map({'regular': 1, 'deviant': 0})

   
    # For matchRequested
    df['MatchedRequest'] = np.where((df.RequestedAmount <= df.OfferedAmount), 'True', 'False')

    df = df.groupby('Case ID').apply(get_duration)
    df = df.reset_index(drop=True)

    
    df['binned_RequestedAmount'] = pd.qcut(df['RequestedAmount'], 5, labels=['0-5000', '5001-10000',
                                                                             '10001-15000', '15001-25000', '25000+'])
    df['binned_RequestedAmount'] = df['binned_RequestedAmount'].astype(str)

    df['binned_duration'] = pd.qcut(df['durationDays'], 5, labels=['0-8', '9-13', '14-22', '23-30', '30+'])
    df['binned_duration'] = df['binned_duration'].astype(str)

    df['binned_NoOfTerms'] = pd.qcut(df['NumberOfTerms'], 5, labels=['6-48', '49-60', '61-96', '97-120', '120+'])
    df['binned_NoOfTerms'] = df['binned_NoOfTerms'].astype(str)

    df['binned_CreditScore'] = pd.qcut(df['CreditScore'], 2, labels=['low', 'high'])
    df['binned_CreditScore'] = df['binned_CreditScore'].astype(str)

    df['binned_MonthlyCost'] = pd.qcut(df['MonthlyCost'], 5, labels=['40-148', '149-200', '201-270',
                                                                     '271-388', '388+'])
    df['binned_MonthlyCost'] = df['binned_MonthlyCost'].astype(str)

    df['binned_FirstWithdrawalAmount'] = pd.qcut(df['FirstWithdrawalAmount'], 3,
                                                 labels=['0-7499', '7500-9895', '9896-75000'])
    df['binned_FirstWithdrawalAmount'] = df['binned_FirstWithdrawalAmount'].astype(str)

    df['binned_NumberOfOffers'] = pd.cut(df['NumberOfOffers'], [1, 2, 3, 11],
                                         include_lowest=True, right=False, labels=['1', '2', '3+'])
    df['binned_NumberOfOffers'] = df['binned_NumberOfOffers'].astype(str)

    df = df.groupby('Case ID').apply(keep_last)
    df = df.reset_index(drop=True)

    # lower case
    column = ['ApplicationType', 'LoanGoal', 'MatchedRequest']
    for col in column:
        df[col] = df[col].str.lower()

    return df


def keep_last(group):
    return group.tail(1)


def get_duration(gr):
    df = pd.DataFrame(gr)
    if len(df[(df["Activity"] == "A_Denied") | (df["Activity"] == "A_Cancelled") | (
            df["Activity"] == "A_Pending")]) > 0:
        try:
            df['new_date'] = [d for d in df['time:timestamp']]
        except:
            #%Y-%m-%d%H:%M:%S.%f
            df['new_date'] = [datetime.strptime(d, '%Y-%m-%d %H:%M:%S.%f+Z') for d in df['time:timestamp']]
            

        first_dt = df[df['Activity'] == 'O_Create Offer']['new_date']
        last_dt = \
            df[(df["Activity"] == "A_Denied") | (df["Activity"] == "A_Cancelled") | (df["Activity"] == "A_Pending")][
                'new_date']

        first_dt = first_dt[first_dt.index.values[0]]
        # print(last_dt)
        last_dt = last_dt[last_dt.index.values[0]]

        d1 = parse(str(first_dt))
        d2 = parse(str(last_dt))

        delta_days = (d2 - d1).days
        # print(delta_days,'\n')
        df['durationDays'] = delta_days
        return df

In [2]:
datetime.strptime('1999-10-12 12:1:2', '%Y-%m-%d %H:%M:%S')

datetime.datetime(1999, 10, 12, 12, 1, 2)

In [3]:
df = pd.read_csv("DahnaLoanApplication_prepared.csv", sep=';')
df.head()

,Case ID,NumberOfOffers,Activity,LoanGoal,ApplicationType,RequestedAmount,Action,org:resource,lifecycle:transition,time:timestamp,EventID,EventOrigin,MonthlyCost,Selected,FirstWithdrawalAmount,Accepted,CreditScore,NumberOfTerms,OfferedAmount,WIP,timesincemidnight,month,weekday,hour,timesincelastevent,timesincecasestart,event_nr,wip,label
0,Application_1000158214,1,A_Create Application,Home improvement,New credit,12500.0,Created,User_1,complete,2016-02-06 10:14:26+00:00,Application_1000158214,Application,0.0,missing,0.0,missing,0.0,0.0,0.0,4884,614,2,5,10,0.000000,0.000000,1,4884,regular
1,Application_1000158214,1,A_Submitted,Home improvement,New credit,12500.0,statechange,User_1,complete,2016-02-06 10:14:26+00:00,ApplState_277536765,Application,0.0,missing,0.0,missing,0.0,0.0,0.0,4884,614,2,5,10,0.000000,0.000000,2,4884,regular
2,Application_1000158214,1,W_Handle leads,Home improvement,New credit,12500.0,Created,User_1,schedule,2016-02-06 10:14:27+00:00,Workitem_176988109,Workflow,0.0,missing,0.0,missing,0.0,0.0,0.0,4884,614,2,5,10,0.016667,0.016667,3,4884,regular
3,Application_1000158214,1,W_Handle leads,Home improvement,New credit,12500.0,Deleted,User_1,withdraw,2016-02-06 10:15:36+00:00,Workitem_895982768,Workflow,0.0,missing,0.0,missing,0.0,0.0,0.0,4885,615,2,5,10,0.000000,1.166667,4,4885,regular
4,Application_1000158214,1,W_Complete application,Home improvement,New credit,12500.0,Created,User_1,schedule,2016-02-06 10:15:36+00:00,Workitem_1409452454,Workflow,0.0,missing,0.0,missing,0.0,0.0,0.0,4885,615,2,5,10,0.000000,1.166667,5,4885,regular


In [4]:
df['time:timestamp'] = pd.to_datetime(df['time:timestamp'], infer_datetime_format=True)
df

,Case ID,NumberOfOffers,Activity,LoanGoal,ApplicationType,RequestedAmount,Action,org:resource,lifecycle:transition,time:timestamp,EventID,EventOrigin,MonthlyCost,Selected,FirstWithdrawalAmount,Accepted,CreditScore,NumberOfTerms,OfferedAmount,WIP,timesincemidnight,month,weekday,hour,timesincelastevent,timesincecasestart,event_nr,wip,label
0,Application_1000158214,1,A_Create Application,Home improvement,New credit,12500.0,Created,User_1,complete,2016-02-06 10:14:26+00:00,Application_1000158214,Application,0.00,missing,0.0,missing,0.0,0.0,0.0,4884,614,2,5,10,0.000000,0.000000,1,4884,regular
1,Application_1000158214,1,A_Submitted,Home improvement,New credit,12500.0,statechange,User_1,complete,2016-02-06 10:14:26+00:00,ApplState_277536765,Application,0.00,missing,0.0,missing,0.0,0.0,0.0,4884,614,2,5,10,0.000000,0.000000,2,4884,regular
2,Application_1000158214,1,W_Handle leads,Home improvement,New credit,12500.0,Created,User_1,schedule,2016-02-06 10:14:27+00:00,Workitem_176988109,Workflow,0.00,missing,0.0,missing,0.0,0.0,0.0,4884,614,2,5,10,0.016667,0.016667,3,4884,regular
3,Application_1000158214,1,W_Handle leads,Home improvement,New credit,12500.0,Deleted,User_1,withdraw,2016-02-06 10:15:36+00:00,Workitem_895982768,Workflow,0.00,missing,0.0,missing,0.0,0.0,0.0,4885,615,2,5,10,0.000000,1.166667,4,4885,regular
4,Application_1000158214,1,W_Complete application,Home improvement,New credit,12500.0,Created,User_1,schedule,2016-02-06 10:15:36+00:00,Workitem_1409452454,Workflow,0.00,missing,0.0,missing,0.0,0.0,0.0,4885,615,2,5,10,0.000000,1.166667,5,4885,regular
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
761367,Application_999993812,1,W_Call incomplete files,Caravan / Camper,New credit,30000.0,Obtained,User_41,resume,2016-10-20 08:19:28+00:00,Workitem_1102967432,Workflow,349.13,True,30000.0,True,877.0,102.0,30000.0,8562,499,10,3,8,1174.883333,189742.400000,31,8562,regular
761368,Application_999993812,1,W_Call incomplete files,Caravan / Camper,New credit,30000.0,Released,User_41,suspend,2016-10-20 08:21:59+00:00,Workitem_454743075,Workflow,349.13,True,30000.0,True,877.0,102.0,30000.0,8562,501,10,3,8,2.516667,189744.916667,32,8562,regular
761369,Application_999993812,1,A_Pending,Caravan / Camper,New credit,30000.0,statechange,User_68,complete,2016-10-24 06:24:30+00:00,ApplState_1253134444,Application,349.13,True,30000.0,True,877.0,102.0,30000.0,8391,384,10,0,6,0.000000,195387.433333,33,8390,regular
761370,Application_999993812,1,W_Call incomplete files,Caravan / Camper,New credit,30000.0,Deleted,User_68,ate_abort,2016-10-24 06:24:30+00:00,Workitem_1302194036,Workflow,349.13,True,30000.0,True,877.0,102.0,30000.0,8391,384,10,0,6,0.000000,195387.433333,34,8390,regular


In [5]:
res_df = read_prep_data(df)
res_df

/home/mshoush/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:4509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(
/home/mshoush/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:4463: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
<ipython-input-1-f8b316932ba2>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df

,Case ID,NumberOfOffers,Activity,LoanGoal,ApplicationType,RequestedAmount,Action,org:resource,lifecycle:transition,time:timestamp,EventID,EventOrigin,MonthlyCost,Selected,FirstWithdrawalAmount,Accepted,CreditScore,NumberOfTerms,OfferedAmount,WIP,timesincemidnight,month,weekday,hour,timesincelastevent,timesincecasestart,event_nr,wip,label,MatchedRequest,new_date,durationDays,binned_RequestedAmount,binned_duration,binned_NoOfTerms,binned_CreditScore,binned_MonthlyCost,binned_FirstWithdrawalAmount,binned_NumberOfOffers
0,Application_1000158214,1,O_Accepted,home improvement,new credit,12500.0,statechange,User_90,complete,2016-10-06 11:02:01+00:00,OfferState_974301742,Offer,250.00,1,9252.0,True,929.0,57.0,12500.0,8999,662,10,3,11,43265.883333,349967.583333,25,8998,1,true,2016-10-06 11:02:01+00:00,122,10001-15000,23-30,49-60,high,201-270,7500-9895,1
1,Application_1000311556,1,W_Call after offers,car,new credit,45000.0,Released,User_16,suspend,2016-08-04 14:03:40+00:00,Workitem_1508760141,Workflow,500.00,0,9252.0,True,906.0,111.0,45000.0,11364,843,8,3,14,0.350000,175567.050000,18,11363,0,true,2016-08-04 14:03:40+00:00,30,25000+,14-22,97-120,low,388+,7500-9895,1
2,Application_1000339879,1,O_Accepted,existing loan takeover,new credit,37500.0,statechange,User_87,complete,2016-03-30 09:11:48+00:00,OfferState_1469594792,Offer,375.00,1,9252.0,True,798.0,126.0,37500.0,8844,551,3,2,9,1469.383333,18494.633333,51,8843,1,true,2016-03-30 09:11:48+00:00,12,25000+,9-13,120+,low,271-388,7500-9895,1
3,Application_100034150,1,O_Accepted,existing loan takeover,new credit,5000.0,statechange,User_119,complete,2016-08-03 11:35:09+00:00,OfferState_836746104,Offer,56.30,1,577.0,False,824.0,120.0,5000.0,11479,695,8,2,11,17.000000,306900.266667,55,11478,1,true,2016-08-03 11:35:09+00:00,158,0-5000,30+,97-120,low,40-148,0-7499,1
4,Application_1000647090,1,W_Call after offers,existing loan takeover,new credit,6500.0,Released,User_34,suspend,2016-04-03 09:48:07+00:00,Workitem_1296758509,Workflow,125.00,1,2000.0,True,906.0,66.0,7000.0,8780,588,4,6,9,0.683333,50393.500000,33,8779,0,true,2016-04-03 09:48:07+00:00,29,5001-10000,9-13,61-96,low,40-148,0-7499,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15992,Application_998722780,2,W_Validate application,existing loan takeover,new credit,21000.0,Deleted,User_75,ate_abort,2016-11-25 09:44:55+00:00,Workitem_405693756,Workflow,500.00,0,9252.0,False,906.0,46.0,21000.0,4798,584,11,4,9,121.200000,372503.883333,44,4797,1,true,2016-11-25 09:44:55+00:00,258,15001-25000,30+,6-48,low,388+,7500-9895,2
15993,Application_998917442,1,W_Call after offers,home improvement,new credit,50000.0,Deleted,User_1,ate_abort,2016-10-31 07:00:23+00:00,Workitem_1598261277,Workflow,539.66,0,41000.0,False,906.0,120.0,53000.0,7857,420,10,0,7,44169.616667,293064.933333,22,7855,0,true,2016-10-31 07:00:23+00:00,30,25000+,14-22,97-120,low,388+,9896-75000,1
15994,Application_999487618,1,A_Pending,existing loan takeover,new credit,20000.0,statechange,User_100,complete,2016-08-29 12:06:27+00:00,ApplState_1514297521,Application,273.49,1,20000.0,True,1064.0,85.0,20000.0,11450,726,8,0,12,0.016667,16908.733333,29,11449,1,true,2016-08-29 12:06:27+00:00,10,15001-25000,0-8,61-96,high,201-270,9896-75000,1
15995,Application_999544538,5,W_Call incomplete files,car,new credit,50000.0,Released,User_14,suspend,2016-11-07 08:13:28+00:00,Workitem_2001406271,Workflow,509.11,0,9252.0,True,906.0,120.0,50000.0,6199,493,11,0,8,1.600000,439033.716667,102,6198,1,true,2016-11-07 08:13:28+00:00,58,25000+,23-30,97-120,low,388+,7500-9895,3+


In [6]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
from actionrules.actionRulesDiscovery import ActionRulesDiscovery


def actionDiscovery(data, stabel_cols, flexible_cols, consequent_col):
    stable_cols = stabel_cols
    flexible_cols = flexible_cols
    consequent_col = consequent_col
    # print(f"Stable_cols: {stabel_cols}\n")
    # print(f"flexible_cols: {flexible_cols}\n")

    actionRulesDiscovery = ActionRulesDiscovery()
    actionRulesDiscovery.load_pandas(data)
    actionRulesDiscovery.fit(stable_attributes=stable_cols,
                             flexible_attributes=flexible_cols,
                             consequent=consequent_col,
                             conf=55,
                             supp=3,
                             desired_classes=["1"],  # fail to success / not survived to survived
                             max_stable_attributes=30,
                             max_flexible_attributes=30,
                             )
    rules = actionRulesDiscovery.get_action_rules()
    rules_representation = actionRulesDiscovery.get_action_rules_representation()
    length = len(actionRulesDiscovery.get_action_rules_representation())

    return rules, length, rules_representation


def interpretRules(rules, length, rules_representation):
    print(f"The number of discovered rules are: {len(rules)}\n")
    # i = 1
    [print(f"Rule: {rule}\n") for rule in rules_representation]
    # print("+++++++++++++++++++++++++++++++++++++++++++++++++++")

    supp = []
    conf = []
    uplift = []
    if not rules:
        print("No Discovered Rules")
    else:
        [supp.append(rules[i][1][2]) for i in range(length)]
        [conf.append(rules[i][2][2]) for i in range(length)]
        [uplift.append(rules[i][3]) for i in range(length)]

        print(f"max supp: {max(supp)}")
        print(f"max conf: {max(conf)}")
        print(f"max uplift: {max(uplift)}")
        # print(f"================ Finish=================\n\n")

    return uplift


def get_unique_actions(r_copy):
    df_u = pd.DataFrame()
    rule_id = 0
    for rule in r_copy:
        tmp = pd.DataFrame()
        j = 0
        for i in rule:
            tmp.at[0, j] = str(i)
            j += 1
        rule_id += 1
        df_u = df_u.append(tmp, ignore_index=True)

    df_u.columns = ["a", "b", "c", "d"]
    df_u["a_new"] = "0"

    for index, row in df_u.iterrows():
        lst = [i for i in str(row["a"]).split("]],")]
        row["a_new"] = lst

    df2 = pd.concat([df_u, df_u["a_new"].apply(pd.Series)], axis=1)
    df2.columns = ["a", "b", "c", "d", "a_new", "stable", "flex", "outcome"]
    rules_processed = df2
    tmp = []
    for _, rule in rules_processed.groupby("flex"):
        #     tmp.append(rule["a"].iloc[0])
        tmp.append(rule.index[0])

    r_copy2 = []
    for i in tmp:
        r_copy2.append(r_copy[i])

    return r_copy2

In [7]:
import pandas as pd
import numpy as np


def filter_log(df_, rule):
    # select cases that satisfy the treatment conditions
    mask = 1
    for i in range(len(rule[0][1])):
        mask = mask & ((df_[rule[0][1][i][0]] == rule[0][1][i][1][0]) | (df_[rule[0][1][i][0]] == rule[0][1][i][1][1]))

    df2 = df_[mask].copy()

    # assign treatment flag
    mask = np.ones(df2.shape[0], dtype=int)
    for i in range(len(rule[0][1])):
        # df2['Treatment'] = np.where((df2[rule[0][1][i][0]] == rule[0][1][i][1][1]), 1, 0)
        this_mask = np.where(df2[rule[0][1][i][0]].astype(str) == rule[0][1][i][1][1], 1, 0)
        mask = mask * this_mask
    df2['Treatment'] = mask.astype(str)

    return df2


def process_data(df_rule):
    num_cols = ['CreditScore', 'MonthlyCost', 'NumberOfOffers', 'NoOfTerms', 'FirstWithdrawalAmount',
                'Treatment', 'Selected', 'label']
    cat_cols = ['ApplicationType', 'LoanGoal']

    for col in df_rule.columns:
        if (col not in num_cols) and (col not in cat_cols):
            df_rule = df_rule.drop(col, axis=1)

    # one hot encoding of categorical data
    tmp = pd.DataFrame()
    for col in cat_cols:
        tmp = pd.concat([tmp, pd.get_dummies(df_rule[col])], axis=1)
        df_rule = df_rule.drop(col, axis=1)

    df3 = pd.concat([df_rule, tmp], axis=1)

    return df3

In [8]:
from causalml.inference.tree import UpliftTreeClassifier
from causalml.inference.tree import uplift_tree_string, uplift_tree_plot
from sklearn.model_selection import train_test_split
#from log_processing import filter_log, process_data


def create_uplift_tree(df, r_copy2):
    for rule in r_copy2:
        print(rule)
        print("index:", r_copy2.index(rule))
        df_rule = filter_log(df, rule)
        print(df_rule.shape)
        df3 = process_data(df_rule)
        df_train, df_test = train_test_split(df3, test_size=0.2)
        features = df_train
        for col in ['label', 'Treatment']:
            features = features.drop(col, axis=1)

        # Train uplift tree
        uplift_model = UpliftTreeClassifier(max_depth=5, min_samples_leaf=200, min_samples_treatment=50,
                                            n_reg=100, evaluationFunction='KL', control_name='0')

        # Train uplift random forest
        # uplift_model = UpliftRandomForestClassifier(control_name='0')
        uplift_model.fit(features.values, treatment=df_train['Treatment'].values, y=df_train['label'].values)

        # Print uplift tree as a string
        result = uplift_tree_string(uplift_model.fitted_uplift_tree, features.columns.values.tolist())

        # save uplift tree as png
        graph = uplift_tree_plot(uplift_model.fitted_uplift_tree, features.columns.values.tolist())
        with open("tree2_%s.png" % r_copy2.index(rule), "wb") as png:
            png.write(graph.create_png())
        # display(Image(graph.create_png()))

    #   y_pred = uplift_model.predict(df_test.values)

    #   # Put the predictions to a DataFrame for a neater presentation
    #   result = pd.DataFrame(y_pred, columns=uplift_model.classes_)
    #   print(result)

In [9]:
res_df.columns

Index(['Case ID', 'NumberOfOffers', 'Activity', 'LoanGoal', 'ApplicationType',
       'RequestedAmount', 'Action', 'org:resource', 'lifecycle:transition',
       'time:timestamp', 'EventID', 'EventOrigin', 'MonthlyCost', 'Selected',
       'FirstWithdrawalAmount', 'Accepted', 'CreditScore', 'NumberOfTerms',
       'OfferedAmount', 'WIP', 'timesincemidnight', 'month', 'weekday', 'hour',
       'timesincelastevent', 'timesincecasestart', 'event_nr', 'wip', 'label',
       'MatchedRequest', 'new_date', 'durationDays', 'binned_RequestedAmount',
       'binned_duration', 'binned_NoOfTerms', 'binned_CreditScore',
       'binned_MonthlyCost', 'binned_FirstWithdrawalAmount',
       'binned_NumberOfOffers'],
      dtype='object')

In [11]:
#from data_prep import read_prep_data
#from action_rules import actionDiscovery, interpretRules, get_unique_actions
#from uplift_tree import create_uplift_tree

data = 'BPIC17_O_Accepted.csv'
df = res_df

consequent_col = 'label'

st_col = ['ApplicationType', 'binned_CreditScore', 'LoanGoal', 'binned_RequestedAmount']
flex_col = ['binned_FirstWithdrawalAmount', 'binned_MonthlyCost', 
            'binned_NoOfTerms', 'MatchedRequest', 'binned_NumberOfOffers']

uplift = []
r_list = []
rep_list = []
print("=================Start============================================\n")
print(f"Stable cols: {st_col}\nFlexible_cols: {flex_col}\n")
rules, length, rules_representation = actionDiscovery(df, st_col, flex_col, consequent_col)
uplift.extend(interpretRules(rules, length, rules_representation))
rep_list.extend(rules_representation)
r_list.extend(rules)
print("===============================Finish================================\n")

r_copy2 = get_unique_actions(r_list)
create_uplift_tree(df, r_copy2)

=================Start============================================

Stable cols: ['ApplicationType', 'binned_CreditScore', 'LoanGoal', 'binned_RequestedAmount']
Flexible_cols: ['binned_FirstWithdrawalAmount', 'binned_MonthlyCost', 'binned_NoOfTerms', 'MatchedRequest', 'binned_NumberOfOffers']

The number of discovered rules are: 25

Rule: r = [(binned_CreditScore: low) ∧ (ApplicationType: new credit) ∧ (binned_NumberOfOffers: 1 → 2)  ∧ (binned_NoOfTerms: 49-60 → 97-120) ] ⇒ [label: 0 → 1] with support: 0.034443958242170405, confidence: 0.4322354134180291 and uplift: 0.042314633680989325.

Rule: r = [(binned_CreditScore: low) ∧ (ApplicationType: new credit) ∧ (binned_NumberOfOffers: 1 → 2)  ∧ (binned_NoOfTerms: 6-48 → 97-120) ] ⇒ [label: 0 → 1] with support: 0.034443958242170405, confidence: 0.44852675035791056 and uplift: 0.047273331945777375.

Rule: r = [(binned_CreditScore: low) ∧ (ApplicationType: new credit) ∧ (binned_NumberOfOffers: 1 → 2)  ∧ (binned_NoOfTerms: 61-96 → 97-120) ] ⇒

[[[['ApplicationType', ('new credit',)]], [['binned_FirstWithdrawalAmount', ('7500-9895', '0-7499')], ['binned_NoOfTerms', ('49-60', '97-120')]], ['label', ['0', '1']]], [0.04975932987435144, 0.0506970056885666, 0.04975932987435144], [0.5772298767222626, 0.6116138763197587, 0.3530418024296795], 0.016279023282174614]
index: 2
(5042, 40)
MonthlyCost >= 97.4?
yes -> CreditScore >= 961.0?
		yes -> {'0': 1.0, '1': 1.0}
		no  -> MonthlyCost >= 431.94899999999996?
				yes -> {'0': 0.504587, '1': 0.563025}
				no  -> existing loan takeover >= 0.1?
						yes -> {'0': 0.406646, '1': 0.61949}
						no  -> {'0': 0.376601, '1': 0.65873}
no  -> {'0': 0.396226, '1': 0.454545}
[[[['ApplicationType', ('new credit',)]], [['binned_FirstWithdrawalAmount', ('7500-9895', '0-7499')], ['binned_NoOfTerms', ('6-48', '97-120')]], ['label', ['0', '1']]], [0.04938425954866538, 0.0506970056885666, 0.04938425954866538], [0.6294820717131474, 0.6116138763197587, 0.38499996995427044], 0.018914509894436278]
index: 3
(54

[[[['binned_CreditScore', ('low',)], ['ApplicationType', ('new credit',)]], [['binned_NumberOfOffers', ('1', '2')], ['binned_NoOfTerms', ('120+', '97-120')]], ['label', ['0', '1']]], [0.04188285303494405, 0.034443958242170405, 0.034443958242170405], [0.7403314917127072, 0.5745568300312826, 0.42536251505078376], 0.017814210863181267]
index: 13
(5676, 40)
Selected >= 0.1?
yes -> CreditScore >= 906.0?
		yes -> FirstWithdrawalAmount >= 15000.0?
				yes -> {'0': 0.725173, '1': 0.716981}
				no  -> {'0': 0.607798, '1': 0.701493}
		no  -> CreditScore >= 838.5?
				yes -> {'0': 1.0, '1': 1.0}
				no  -> {'0': 1.0, '1': 1.0}
no  -> FirstWithdrawalAmount >= 14843.999999999989?
		yes -> FirstWithdrawalAmount >= 25000.0?
				yes -> {'0': 0.004525, '1': 0.568807}
				no  -> {'0': 0.016575, '1': 0.512397}
		no  -> FirstWithdrawalAmount >= 9252.0?
				yes -> {'0': 0.107317, '1': 0.507692}
				no  -> existing loan takeover >= 0.1?
						yes -> {'0': 0.008621, '1': 0.581197}
						no  -> {'0': 0.056738